In [1]:
! pip install   datasets  sentence_transformers openpyxl --q
! pip install   transformers[torch] --q
! pip install   accelerate==0.27.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import uuid

from huggingface_hub import login
from google.colab import userdata

from datasets import Dataset, load_dataset, DatasetDict, concatenate_datasets
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator
)
from sentence_transformers.util import cos_sim


login(token=userdata.get('hf_token'))
os.environ["OPENAI_API_KEY"] =  userdata.get('openai_key')
os.environ["WANDB_API_KEY"]  =  userdata.get('wand_key')

# *Mount Google Drive and Read dataset*

In [3]:
from google.colab import drive
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/MyDrive/dataset/train_triplet_dataset.csv')
df_test  = pd.read_csv('/content/drive/MyDrive/dataset/test_triplet_dataset.csv')

Mounted at /content/drive


# *Flag Embedding Dataset*

In [4]:

def format_text(query, positive_text, negative_text):
    query = f"Represent this sentence for searching relevant passages: {query}"
    return query, positive_text, negative_text


def format_row(row):
    anchor, pos, neg = format_text(row["anchor"], row["positive"], row["negative"])
    return pd.Series([anchor, pos, neg], index=["anchor", "positive", "negative"])

# Apply the formatting to the entire DataFrame
df_train_prefix = df_train.apply(format_row, axis=1)
df_test_prefix = df_test.apply(format_row, axis=1)

dataset_train_prefix = Dataset.from_pandas(df_train_prefix)
dataset_test_prefix  = Dataset.from_pandas(df_test_prefix)

dataset_train_prefix  = dataset_train_prefix.add_column("id", [str(uuid.uuid4()) for _ in range(len(dataset_train_prefix))])
dataset_test_prefix   = dataset_test_prefix.add_column("id", [str(uuid.uuid4()) for _ in range(len(dataset_test_prefix))])

dataset_train_prefix.to_json("train_dataset_prefix.json", orient="records")
dataset_test_prefix.to_json("test_dataset_prefix.json", orient="records")

corpus_dataset_prefix = concatenate_datasets([dataset_train_prefix, dataset_test_prefix])

corpus_prefix = dict(
    zip(corpus_dataset_prefix["id"], corpus_dataset_prefix["positive"])
)
queries_with_prefix = dict(
    zip(dataset_test_prefix["id"], dataset_test_prefix["anchor"])
)


Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

# *Convert to hugging Face Dataset*

In [5]:
dataset_train        = Dataset.from_pandas(df_train)
dataset_test         = Dataset.from_pandas(df_test)

dataset_train = dataset_train.add_column("id", [str(uuid.uuid4()) for _ in range(len(dataset_train))])
dataset_test  = dataset_test.add_column("id", [str(uuid.uuid4()) for _ in range(len(dataset_test))])

# save datasets to disk
dataset_train.to_json("train_dataset.json", orient="records")
dataset_test.to_json("test_dataset.json", orient="records")

corpus_dataset = concatenate_datasets([dataset_train, dataset_test])

corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
)
queries = dict(
    zip(dataset_test["id"], dataset_test["anchor"])
)


Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

# ***Create baseline and evaluate pretrained model***

In [ ]:

# test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
# train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")

# test_dataset_prefix = load_dataset("json", data_files="test_dataset_prefix.json", split="train")
# train_dataset_prefix = load_dataset("json", data_files="train_dataset_prefix.json", split="train")



# for queries with prefix instruction


# df = pd.DataFrame(corpus_dataset)
# df.to_excel('/content/drive/MyDrive/dataset/corpus.xlsx', index=False)


In [6]:
def prepare_evaluator(query_set, corpus_set):
  relevant_docs = {}
  for q_id in query_set:
      relevant_docs[q_id] = [q_id]

  ir_evaluator = InformationRetrievalEvaluator(
          queries=query_set,
          corpus=corpus_set,
          relevant_docs=relevant_docs,
          show_progress_bar=True,
          accuracy_at_k=[1, 3, 5, 7, 10],
          precision_recall_at_k=[1, 3, 5,7,10],
          map_at_k=[100],
          mrr_at_k=[10],
          ndcg_at_k=[10],
          name=f"ir_evaluator",
          score_functions={"cosine": cos_sim}
  )
  return ir_evaluator


def call_evaluator(model, is_flag_embedding):
  if is_flag_embedding:
    print(f'call prepare_evaluator(queries_with_prefix)')
    ir_evaluator = prepare_evaluator(queries_with_prefix, corpus_prefix)
  else:
    print(f'call prepare_evaluator(queries)')
    ir_evaluator = prepare_evaluator(queries, corpus)

  results = ir_evaluator(model)
  return results

# *Pre Train Evaluation*

In [7]:

import json
def evaluate_model(model_id_or_path, is_pretrained=True, is_flag_embedding=False):

    try:
        model = SentenceTransformer(model_id_or_path, device="cuda" if torch.cuda.is_available() else "cpu")
    except Exception as e:
        print(f"Error loading the model: {e}")
        return
    try:
      results = call_evaluator(model, is_flag_embedding)

    except Exception as e:
      print(f"Error during evaluation: {e}")
      return

    results_filepath = f"/content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_{'pretrained' if is_pretrained else 'finetuned'}_{'prefix' if is_flag_embedding else ''}.txt"
    with open(results_filepath, "w") as f:
      json.dump(results, f)

    print(f"Evaluation results saved to: {results_filepath}")

    for metric, value in results.items():
      print(f"{metric}: {value:.4f}")



In [14]:
'''
    Call model evaluation for InformationRetrieval on test data
    @param model_id_or_path:
    @param is_pretrained: false|true (false is for finetuned model)
    @param is_flag_embedding: false|true (true is for flag embedding test dataset)

    @output:

    /content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_pretrained.txt
    /content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_pretrained_prefix.txt
    /content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_finetuned.txt
    /content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_finetuned_prefix.txt
'''

# evaluate_model("BAAI/bge-large-en-v1.5",True,False)  # call pretrained model evaluation without prefix query instruction
# evaluate_model("BAAI/bge-large-en-v1.5",True,True) # call pretrained model evaluation with prefix query instruction
# evaluate_model("sandeep-aggarwal/bge-large-en-v1.5-finetuned_v8",False,False) # call finetuned model evaluation without prefix query instruction
evaluate_model("sandeep-aggarwal/bge-large-en-v1.5-finetuned_v8",False,True) # call finetuned model evaluation with prefix query instruction
# evaluate_model("sandeep-aggarwal/bge-large-en-v1.5-finetuned_flag_embedding_v2",False,True) # call finetuned model evaluation with prefix query instruction

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

...

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

call prepare_evaluator(queries_with_prefix)


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:31<00:00, 31.70s/it]

Evaluation results saved to: /content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_finetuned_prefix.txt
ir_evaluator_cosine_accuracy@1: 0.2971
ir_evaluator_cosine_accuracy@3: 0.4425
ir_evaluator_cosine_accuracy@5: 0.5021
ir_evaluator_cosine_accuracy@7: 0.5387
ir_evaluator_cosine_accuracy@10: 0.5858
ir_evaluator_cosine_precision@1: 0.2971
ir_evaluator_cosine_precision@3: 0.1475
ir_evaluator_cosine_precision@5: 0.1004
ir_evaluator_cosine_precision@7: 0.0770
ir_evaluator_cosine_precision@10: 0.0586
ir_evaluator_cosine_recall@1: 0.2971
ir_evaluator_cosine_recall@3: 0.4425
ir_evaluator_cosine_recall@5: 0.5021
ir_evaluator_cosine_recall@7: 0.5387
ir_evaluator_cosine_recall@10: 0.5858
ir_evaluator_cosine_ndcg@10: 0.4340
ir_evaluator_cosine_mrr@10: 0.3864
ir_evaluator_cosine_map@100: 0.3962


# *Initialize model*






In [ ]:
from sentence_transformers import SentenceTransformerModelCardData, SentenceTransformer

# Hugging Face model ID: https://huggingface.co/BAAI/bge-base-en-v1.5
model_id = "BAAI/bge-large-en-v1.5"
device = "cuda" if torch.cuda.is_available() else "cpu"
# load model with SDPA for using Flash Attention 2
model = SentenceTransformer(
    model_id,
    device=device,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="BGE large model",
    ),
)
#model = torch.compile(model)
#model.gradient_checkpointing_enable()
print(f"Using device: {device}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Using device: cuda


#*Initialise Loss Function*

In [ ]:
from sentence_transformers.losses import  MultipleNegativesRankingLoss,TripletLoss,ContrastiveLoss

train_loss = TripletLoss(model)
#train_loss = MultipleNegativesRankingLoss(model)


# *Fine Tune embedding model*

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

# load train dataset again
#train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")

# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/bge-large-triplet-v1.5", # output directory and hugging face model ID
    num_train_epochs=100,                         # number of epochs
    per_device_train_batch_size=16,             # train batch size
    gradient_accumulation_steps=16,             # for a global batch size of 512
    per_device_eval_batch_size=10,               # evaluation batch size
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                   # use fused adamw optimizer
    tf32=True,                                  # use tf32 precision
    bf16=True,                                   # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="epoch",                      # evaluate after each epoch
    save_strategy="epoch",                      # save after each epoch
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    run_name="bge-large-en-v1.5-finetuned_flag_embedding_v2",
    metric_for_best_model="eval_ir_evaluator_cosine_ndcg@10"
)

In [ ]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model, # bg-large-en-v1
    args=args,  # training arguments
    train_dataset=dataset_train_prefix.select_columns(
        ["positive", "anchor", "negative"]
    ),
    loss=train_loss,
    evaluator=prepare_evaluator(queries_with_prefix, corpus_prefix),
)

In [ ]:
trainer.train()

wandb: Currently logged in as: ai-model-training (ai-model-training-adobe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


dataset = dataset.select_columns(['anchor', 'positive', 'negative'])


Epoch,Training Loss,Validation Loss,Ir Evaluator Cosine Accuracy@1,Ir Evaluator Cosine Accuracy@3,Ir Evaluator Cosine Accuracy@5,Ir Evaluator Cosine Accuracy@7,Ir Evaluator Cosine Accuracy@10,Ir Evaluator Cosine Precision@1,Ir Evaluator Cosine Precision@3,Ir Evaluator Cosine Precision@5,Ir Evaluator Cosine Precision@7,Ir Evaluator Cosine Precision@10,Ir Evaluator Cosine Recall@1,Ir Evaluator Cosine Recall@3,Ir Evaluator Cosine Recall@5,Ir Evaluator Cosine Recall@7,Ir Evaluator Cosine Recall@10,Ir Evaluator Cosine Ndcg@10,Ir Evaluator Cosine Mrr@10,Ir Evaluator Cosine Map@100
1,77.746700,No log,0.254184,0.376569,0.432008,0.469665,0.508368,0.254184,0.125523,0.086402,0.067095,0.050837,0.254184,0.376569,0.432008,0.469665,0.508368,0.372369,0.329864,0.340822
2,76.022300,No log,0.182008,0.278243,0.328452,0.376569,0.415272,0.182008,0.092748,0.065690,0.053796,0.041527,0.182008,0.278243,0.328452,0.376569,0.415272,0.287503,0.248051,0.258628
3,74.885700,No log,0.230126,0.351464,0.413180,0.458159,0.505230,0.230126,0.117155,0.082636,0.065451,0.050523,0.230126,0.351464,0.413180,0.458159,0.505230,0.355799,0.309448,0.321357
4,72.763100,No log,0.230126,0.367155,0.425732,0.470711,0.511506,0.230126,0.122385,0.085146,0.067244,0.051151,0.230126,0.367155,0.425732,0.470711,0.511506,0.362268,0.315605,0.326377
5,71.613300,No log,0.185146,0.306485,0.361925,0.401674,0.443515,0.185146,0.102162,0.072385,0.057382,0.044351,0.185146,0.306485,0.361925,0.401674,0.443515,0.304904,0.261679,0.271779
6,69.734200,No log,0.156904,0.268828,0.328452,0.361925,0.395397,0.156904,0.089609,0.065690,0.051704,0.039540,0.156904,0.268828,0.328452,0.361925,0.395397,0.267712,0.227763,0.238171
7,68.834600,No log,0.133891,0.228033,0.279289,0.315900,0.352510,0.133891,0.076011,0.055858,0.045129,0.035251,0.133891,0.228033,0.279289,0.315900,0.352510,0.232869,0.195801,0.206026
8,67.266700,No log,0.096234,0.184100,0.240586,0.275105,0.313808,0.096234,0.061367,0.048117,0.039301,0.031381,0.096234,0.184100,0.240586,0.275105,0.313808,0.194397,0.157489,0.167100
9,66.312100,No log,0.085774,0.167364,0.208159,0.236402,0.268828,0.085774,0.055788,0.041632,0.033772,0.026883,0.085774,0.167364,0.208159,0.236402,0.268828,0.169425,0.138611,0.148598
10,65.164200,No log,0.073222,0.149582,0.193515,0.209205,0.233264,0.073222,0.049861,0.038703,0.029886,0.023326,0.073222,0.149582,0.193515,0.209205,0.233264,0.148480,0.121860,0.130578


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:06<00:00,  6.00s/it]


...

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:05<00:00,  5.81s/it]


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:05<00:00,  5.82s/it]


TrainOutput(global_step=1500, training_loss=58.11694327799479, metrics={'train_runtime': 9881.2204, 'train_samples_per_second': 38.71, 'train_steps_per_second': 0.152, 'total_flos': 0.0, 'train_loss': 58.11694327799479, 'epoch': 100.0})

In [ ]:
trainer.save_model()
trainer.model.push_to_hub("bge-large-en-v1.5-finetuned_flag_embedding_v2",private=True)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/sandeep-aggarwal/bge-large-en-v1.5-finetuned_flag_embedding_v2/commit/cc73f61c1ed557579e73c3b9bea92f1489f020cf'

#*Run Post Fine Tuning Evaluation*

In [ ]:
# Evaluate the model
#evaluate_model("sandeep-aggarwal/bge-base-en-v1.5-finetuned_v22",False)
evaluate_model(args.output_dir,False,True)

call prepare_evaluator(queries_with_prefix)


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:07<00:00,  7.74s/it]


Evaluation results saved to: /content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_finetuned_prefix.txt
ir_evaluator_cosine_accuracy@1: 0.2814
ir_evaluator_cosine_accuracy@3: 0.4111
ir_evaluator_cosine_accuracy@5: 0.4644
ir_evaluator_cosine_accuracy@7: 0.5031
ir_evaluator_cosine_accuracy@10: 0.5471
ir_evaluator_cosine_precision@1: 0.2814
ir_evaluator_cosine_precision@3: 0.1370
ir_evaluator_cosine_precision@5: 0.0929
ir_evaluator_cosine_precision@7: 0.0719
ir_evaluator_cosine_precision@10: 0.0547
ir_evaluator_cosine_recall@1: 0.2814
ir_evaluator_cosine_recall@3: 0.4111
ir_evaluator_cosine_recall@5: 0.4644
ir_evaluator_cosine_recall@7: 0.5031
ir_evaluator_cosine_recall@10: 0.5471
ir_evaluator_cosine_ndcg@10: 0.4057
ir_evaluator_cosine_mrr@10: 0.3615
ir_evaluator_cosine_map@100: 0.3720


# *Display Comparision Metric*

In [15]:
from IPython.display import display
import json
import pandas as pd

with open('/content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_pretrained.txt', 'r') as file:
    pretrained_metrics = json.load(file)

with open('/content/drive/MyDrive/dataset/model/results/bge-large/evaluation_results_finetuned_prefix.txt', 'r') as file:
    finetuned_metrics = json.load(file)

# df_pretrained = pd.DataFrame(list(pretrained_metrics.items()), columns=["Metric", "Pre-trained(without Query Instructions)"])
# df_finetuned = pd.DataFrame(list(finetuned_metrics.items()), columns=["Metric", "Pre-trained(with Query Instructions)"])
# df_comparison = pd.merge(df_pretrained, df_finetuned, on="Metric")
# df_comparison["%Improvement"] = ((df_comparison["Pre-trained(with Query Instructions)"] - df_comparison["Pre-trained(without Query Instructions)"]) / df_comparison["Pre-trained(without Query Instructions)"]) * 100

df_pretrained = pd.DataFrame(list(pretrained_metrics.items()), columns=["Metric", "Pre-Trained"])
df_finetuned = pd.DataFrame(list(finetuned_metrics.items()), columns=["Metric", "Fine-Tuned"])
df_comparison = pd.merge(df_pretrained, df_finetuned, on="Metric")
df_comparison["%Improvement"] = ((df_comparison["Fine-Tuned"] - df_comparison["Pre-Trained"]) / df_comparison["Pre-Trained"]) * 100

display(df_comparison)

,Metric,Pre-Trained,Fine-Tuned,%Improvement
0,ir_evaluator_cosine_accuracy@1,0.276151,0.297071,7.575758
1,ir_evaluator_cosine_accuracy@3,0.417364,0.442469,6.015038
2,ir_evaluator_cosine_accuracy@5,0.475941,0.502092,5.494505
3,ir_evaluator_cosine_accuracy@7,0.510460,0.538703,5.532787
4,ir_evaluator_cosine_accuracy@10,0.555439,0.585774,5.461394
5,ir_evaluator_cosine_precision@1,0.276151,0.297071,7.575758
6,ir_evaluator_cosine_precision@3,0.139121,0.147490,6.015038
7,ir_evaluator_cosine_precision@5,0.095188,0.100418,5.494505
8,ir_evaluator_cosine_precision@7,0.072923,0.076958,5.532787
9,ir_evaluator_cosine_precision@10,0.055544,0.058577,5.461394
